### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/210_TROCAS7_Fusion_Apr2021_PEAKS_152/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/210_TROCAS7_Fusion_Apr2021_PEAKS_152


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 210_TROCAS7_Fusion_Apr2021_PEAKS_152_peptide.csv
 210_TROCAS7_Fusion_Apr2021_PEAKS_152_protein.csv
 210_TROCAS7_Fusion_Apr2021_PEAKS_152_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup210 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/210_TROCAS7_Fusion_Apr2021_PEAKS_152/210_TROCAS7_Fusion_Apr2021_PEAKS_152_peptide.csv")

# remove redundant rows
peaksdb210 = pd.DataFrame.drop_duplicates(peaksdbdup210)

print(peaksdb210.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb210.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb210['#Spec']
del peaksdb210['Accession']
del peaksdb210['PTM']
del peaksdb210['AScore']

mean_length = peaksdb210['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup210))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb210))

#look at the dataframe
peaksdb210.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-210-SMCP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-210-SMCP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.528301886792454
# redundant peaksdb peptides in combined dataframe 106
# nonredundant peaksdb peptides in combined dataframe 106


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,87.84,1881.8350,18,1.0,941.9257,91.00,2010000.0,30,15964,20210407_Trocas7_664_SMCP210_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,73.83,2210.0967,20,0.5,737.7065,94.62,2850000.0,30,16563,20210407_Trocas7_664_SMCP210_DDA_120min_1.raw
2,IDVLEGNEQFINAAK,67.77,1659.8468,15,1.3,830.9318,94.25,3290000.0,30,16498,20210407_Trocas7_664_SMCP210_DDA_120min_1.raw
3,SSGSSYPSLLQC(+57.02)LK,65.81,1525.7446,14,0.6,763.8801,99.06,517000.0,30,17276,20210407_Trocas7_664_SMCP210_DDA_120min_1.raw
4,VLEGNEQFINAAK,51.62,1431.7357,13,1.4,716.8762,70.86,3270000.0,30,12915,20210407_Trocas7_664_SMCP210_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb210['A'] = peaksdb210['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb210['C'] = peaksdb210['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb210['D'] = peaksdb210['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb210['E'] = peaksdb210['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb210['F'] = peaksdb210['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb210['G'] = peaksdb210['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb210['H'] = peaksdb210['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb210 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb210['I'] = peaksdb210['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb210['K'] = peaksdb210['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb210['L'] = peaksdb210['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb210['M'] = peaksdb210['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb210['N'] = peaksdb210['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb210['P'] = peaksdb210['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb210['Q'] = peaksdb210['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb210['R'] = peaksdb210['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb210['S'] = peaksdb210['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb210['T'] = peaksdb210['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb210['V'] = peaksdb210['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb210['W'] = peaksdb210['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb210['Y'] = peaksdb210['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb210['c-carb'] = peaksdb210['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb210['m-oxid'] = peaksdb210['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb210['n-deam'] = peaksdb210['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb210['q-deam'] = peaksdb210['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb210['stripped_peptide'] = peaksdb210['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb210['stripped_length'] = peaksdb210['stripped_peptide'].apply(len)

peaksdb210['NAAF_num.'] = peaksdb210['Area'] / peaksdb210['stripped_length']

# total the number of modifications in sequence
peaksdb210['ptm-total'] = peaksdb210['c-carb'] + peaksdb210['m-oxid'] + peaksdb210['n-deam'] + peaksdb210['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb210['stripped_IL']= peaksdb210['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb210.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB.csv")

# check out the results
peaksdb210.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,87.84,1881.8350,18,1.0,941.9257,91.00,2010000.0,30,15964,...,0,2,0,0,0,SCLISGWGNTK,11,182727.272727,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,73.83,2210.0967,20,0.5,737.7065,94.62,2850000.0,30,16563,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,142500.000000,0,LGEHNLDVLEGNEQFLNAAK
2,IDVLEGNEQFINAAK,67.77,1659.8468,15,1.3,830.9318,94.25,3290000.0,30,16498,...,0,0,0,0,0,IDVLEGNEQFINAAK,15,219333.333333,0,LDVLEGNEQFLNAAK
3,SSGSSYPSLLQC(+57.02)LK,65.81,1525.7446,14,0.6,763.8801,99.06,517000.0,30,17276,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,36928.571429,1,SSGSSYPSLLQCLK
4,VLEGNEQFINAAK,51.62,1431.7357,13,1.4,716.8762,70.86,3270000.0,30,12915,...,0,0,0,0,0,VLEGNEQFINAAK,13,251538.461538,0,VLEGNEQFLNAAK


In [6]:
# keep only stripped peptide I/L and NAAF
db_210 = peaksdb210[['stripped_IL', 'Area', 'NAAF_num.']]

db_210.set_index('stripped_IL')

# write modified dataframe to new txt file
db_210.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/210_MS_T00_GD_PDB_ILnaafs.csv")

db_210.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,2010000.0,182727.272727
1,LGEHNLDVLEGNEQFLNAAK,2850000.0,142500.000000
2,LDVLEGNEQFLNAAK,3290000.0,219333.333333
3,SSGSSYPSLLQCLK,517000.0,36928.571429
4,VLEGNEQFLNAAK,3270000.0,251538.461538


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep210moddup = peaksdb210[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep210dup = peaksdb210[["stripped_peptide"]]

# deduplicate both of these lists
pep210mod = pep210moddup.drop_duplicates()
pep210 = pep210dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 210:", len(pep210moddup))
print("Deduplicated modified PeaksDB peptides in 210:", len(pep210mod))
print("Total stripped PeaksDB peptides in 210:", len(pep210dup))
print("Deduplicated stripped PeaksDB peptides in 210:", len(pep210))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep210.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep210.head()

Total modified PeaksDB peptides in 210: 106
Deduplicated modified PeaksDB peptides in 210: 106
Total stripped PeaksDB peptides in 210: 106
Deduplicated stripped PeaksDB peptides in 210: 97


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,IDVLEGNEQFINAAK
3,SSGSSYPSLLQCLK
4,VLEGNEQFINAAK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb210['A'].sum(),
        'C': peaksdb210['C'].sum(),
        'D': peaksdb210['D'].sum(),
        'E': peaksdb210['E'].sum(),
        'F': peaksdb210['F'].sum(),
        'G': peaksdb210['G'].sum(),
        'H': peaksdb210['H'].sum(),
        'I': peaksdb210['I'].sum(),
        'K': peaksdb210['K'].sum(),
        'L': peaksdb210['L'].sum(),
        'M': peaksdb210['M'].sum(),
        'N': peaksdb210['N'].sum(),
        'P': peaksdb210['P'].sum(),
        'Q': peaksdb210['Q'].sum(),
        'R': peaksdb210['R'].sum(),
        'S': peaksdb210['S'].sum(),
        'T': peaksdb210['T'].sum(),
        'V': peaksdb210['V'].sum(),
        'W': peaksdb210['W'].sum(),
        'Y': peaksdb210['Y'].sum(),
        'c-carb': peaksdb210['c-carb'].sum(),
        'm-oxid': peaksdb210['m-oxid'].sum(),
        'n-deam': peaksdb210['n-deam'].sum(),
        'q-deam': peaksdb210['q-deam'].sum(),
        'Total area': peaksdb210['Area'].sum(),
        'Total length': peaksdb210['stripped_length'].sum()
       }

totalpeaksdb210 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb210['% C w/ carb'] = totalpeaksdb210['c-carb'] / totalpeaksdb210['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb210['% M w/ oxid'] = totalpeaksdb210['m-oxid'] / totalpeaksdb210['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb210['% N w/ deam'] = totalpeaksdb210['n-deam'] / totalpeaksdb210['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb210['% Q w/ deam'] = totalpeaksdb210['q-deam'] / totalpeaksdb210['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb210['NAAF denom.'] = totalpeaksdb210['Total area'] / totalpeaksdb210['Total length']

# write modified dataframe to new txt file
totalpeaksdb210.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_totals.csv")

totalpeaksdb210.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,102,25,48,65,51,96,32,96,58,122,...,2,18,4,739687046.0,1171,1.0,0.25,0.137405,0.090909,631671.260461


In [9]:
# use the calculated NAAF factor (in totalpeaksdb210 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 631671.260461

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb210['NAAF factor'] = (peaksdb210['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb210_NAAF = peaksdb210[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb210_NAAF['A-NAAF20'] = peaksdb210_NAAF['A'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['C-NAAF20'] = peaksdb210_NAAF['C'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['D-NAAF20'] = peaksdb210_NAAF['D'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['E-NAAF20'] = peaksdb210_NAAF['E'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['F-NAAF20'] = peaksdb210_NAAF['F'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['G-NAAF20'] = peaksdb210_NAAF['G'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['H-NAAF20'] = peaksdb210_NAAF['H'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['I-NAAF20'] = peaksdb210_NAAF['I'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['K-NAAF20'] = peaksdb210_NAAF['K'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['L-NAAF20'] = peaksdb210_NAAF['L'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['M-NAAF20'] = peaksdb210_NAAF['M'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['N-NAAF20'] = peaksdb210_NAAF['N'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['P-NAAF20'] = peaksdb210_NAAF['P'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['Q-NAAF20'] = peaksdb210_NAAF['Q'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['R-NAAF20'] = peaksdb210_NAAF['R'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['S-NAAF20'] = peaksdb210_NAAF['S'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['T-NAAF20'] = peaksdb210_NAAF['T'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['V-NAAF20'] = peaksdb210_NAAF['V'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['W-NAAF20'] = peaksdb210_NAAF['W'] * peaksdb210['NAAF factor']
peaksdb210_NAAF['Y-NAAF20'] = peaksdb210_NAAF['Y'] * peaksdb210['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb210_NAAF['ccarb-NAAF20'] = peaksdb210_NAAF['c-carb'] * peaksdb210_NAAF['NAAF factor']
peaksdb210_NAAF['moxid-NAAF20'] = peaksdb210_NAAF['m-oxid'] * peaksdb210_NAAF['NAAF factor']
peaksdb210_NAAF['ndeam-NAAF20'] = peaksdb210_NAAF['n-deam'] * peaksdb210_NAAF['NAAF factor']
peaksdb210_NAAF['qdeam-NAAF20'] = peaksdb210_NAAF['q-deam'] * peaksdb210_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb210_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_naaf.csv")

peaksdb210_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.289276,3,2,0,1,0,3,0,1,...,0.0,0.578552,0.578552,0.000000,0.289276,0.000000,0.578552,0.0,0.0,0.0
1,LGEHNIDVLEGNEQFINAAK,0.225592,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.225592,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,IDVLEGNEQFINAAK,0.347227,2,0,1,2,1,1,0,2,...,0.0,0.000000,0.000000,0.347227,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,SSGSSYPSLLQCLK,0.058462,0,1,0,0,0,1,0,0,...,0.0,0.292308,0.000000,0.000000,0.000000,0.058462,0.058462,0.0,0.0,0.0
4,VLEGNEQFINAAK,0.398211,2,0,0,2,1,1,0,1,...,0.0,0.000000,0.000000,0.398211,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb210 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb210_NAAF['NAAF factor'].sum(),
        'A': peaksdb210_NAAF['A-NAAF20'].sum(),
        'C': peaksdb210_NAAF['C-NAAF20'].sum(),
        'D': peaksdb210_NAAF['D-NAAF20'].sum(),
        'E': peaksdb210_NAAF['E-NAAF20'].sum(),
        'F': peaksdb210_NAAF['F-NAAF20'].sum(),
        'G': peaksdb210_NAAF['G-NAAF20'].sum(),
        'H': peaksdb210_NAAF['H-NAAF20'].sum(),
        'I': peaksdb210_NAAF['I-NAAF20'].sum(),
        'K': peaksdb210_NAAF['K-NAAF20'].sum(),
        'L': peaksdb210_NAAF['L-NAAF20'].sum(),
        'M': peaksdb210_NAAF['M-NAAF20'].sum(),
        'N': peaksdb210_NAAF['N-NAAF20'].sum(),
        'P': peaksdb210_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb210_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb210_NAAF['R-NAAF20'].sum(),
        'S': peaksdb210_NAAF['S-NAAF20'].sum(),
        'T': peaksdb210_NAAF['T-NAAF20'].sum(),
        'V': peaksdb210_NAAF['V-NAAF20'].sum(),
        'W': peaksdb210_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb210_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb210_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb210_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb210_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb210_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb210_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb210_NAAF['% C w/ carb.'] = totalpeaksdb210_NAAF['c-carb'] / totalpeaksdb210_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb210_NAAF['% M w/ oxid'] = totalpeaksdb210_NAAF['m-oxid'] / totalpeaksdb210_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb210_NAAF['% N w/ deam'] = totalpeaksdb210_NAAF['n-deam'] / totalpeaksdb210_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb210_NAAF['% Q w/ deam'] = totalpeaksdb210_NAAF['q-deam'] / totalpeaksdb210_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb210_NAAF['NAAF check'] = totalpeaksdb210_NAAF['NAAF'] / 631671.260461

# write modified dataframe to new txt file, same name + totals
totalpeaksdb210_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/210_MS_T00_GD_PDB/210_MS_T00_GD_PDB_naaf_totals.csv")

totalpeaksdb210_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,138.317151,79.914371,2.292046,7.795621,8.668617,5.682218,12.518672,6.123296,44.819697,3.386515,...,1.281917,2.292046,0.002394,0.808586,0.146849,1.0,0.00626,0.024727,0.032312,0.000219
